In [1]:
import pyodbc
import pandas as pd
from os import getcwd
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from UliPlot.XLSX import auto_adjust_xlsx_column_width

In [2]:
strain_query = '''
SELECT  [StrainGauge].[gid]
      ,[gpn]
      ,[ver] as 'PCB ver'
      ,[toolno]
      ,[station]
      ,[Position]
      ,[TestRecord].judge
      ,[TestRecord].stress
      ,[TestRecord].WQ211
      ,[memo]
      ,[org]
      ,[createdata]
  FROM [SMTvip].[dbo].[StrainGauge]

  Left JOIN [SMTvip].[dbo].[TestRecord] 
  on [StrainGauge].[gid] = [TestRecord].[gid]
  where gpn = '105-04125-00'
  ORDER by profdate DESC
'''

connection_string = "DRIVER={SQL Server};SERVER=T1-PE-SUPPORT;DATABASE=PR_WEB2;UID=pieng;PWD=Q2iT5cwHJW3FH"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
result = pd.read_sql(strain_query, engine)

In [3]:
result.judge = result.judge.map({'OK':1, 'NG':0})
result.sort_values('createdata', ascending=False, inplace=True)
result.head()

,gid,gpn,PCB ver,toolno,station,Position,judge,stress,WQ211,memo,org,createdata
0,10184,105-04125-00,V6,1,ATE,I101,1.0,204.14,350,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
2,10184,105-04125-00,V6,1,ATE,I405,1.0,144.13,200,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
3,10184,105-04125-00,V6,1,ATE,I501,1.0,87.74,250,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
4,10184,105-04125-00,V6,1,ATE,I606,1.0,81.43,200,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
5,10184,105-04125-00,V6,1,ATE,I800,1.0,38.61,250,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00


In [4]:
droped = result.drop_duplicates(subset=['PCB ver','station','Position','memo'])

In [5]:
droped

,gid,gpn,PCB ver,toolno,station,Position,judge,stress,WQ211,memo,org,createdata
0,10184,105-04125-00,V6,1,ATE,I101,1.0,204.14,350,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
2,10184,105-04125-00,V6,1,ATE,I405,1.0,144.13,200,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
3,10184,105-04125-00,V6,1,ATE,I501,1.0,87.74,250,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
4,10184,105-04125-00,V6,1,ATE,I606,1.0,81.43,200,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
5,10184,105-04125-00,V6,1,ATE,I800,1.0,38.61,250,"ATE2.0-ECG治具,延用V4線板,驗證治具:1.2.5.6",T2,2021-07-30 14:06:00
...,...,...,...,...,...,...,...,...,...,...,...,...
192,7875,105-04125-00,V1,無,組裝測試,I706,1.0,124.43,200,Screw(第1片):鎖第1.2顆螺絲紐力0.6kg第3顆0.45kg第4顆0.7kg,T1,2020-05-11 16:13:00
194,7875,105-04125-00,V1,無,組裝測試,I801,0.0,364.17,300,Screw(第1片):鎖第1.2顆螺絲紐力0.6kg第3顆0.45kg第4顆0.7kg,T1,2020-05-11 16:13:00
191,7875,105-04125-00,V1,無,組裝測試,I403,0.0,780.38,350,Screw(第1片):鎖第1.2顆螺絲紐力0.6kg第3顆0.45kg第4顆0.7kg,T1,2020-05-11 16:13:00
190,7875,105-04125-00,V1,無,組裝測試,I101,1.0,131.55,300,Screw(第1片):鎖第1.2顆螺絲紐力0.6kg第3顆0.45kg第4顆0.7kg,T1,2020-05-11 16:13:00


In [38]:
pivot = droped.groupby(['station','memo','PCB ver','Position'])['judge'].sum().unstack('Position').sort_index(level=['station','PCB ver'],ascending=False)